In [ ]:
!unzip '/content/drive/MyDrive/machineHack/uhack_sentiments_20_decode_code_words/Participants_Data_DCW.zip'

Archive:  /content/drive/MyDrive/machineHack/uhack_sentiments_20_decode_code_words/Participants_Data_DCW.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('train.csv')
train.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1
3,3,Work great easy to use . No issues at all with...,0,0,0,0,0,1,0,0,0,0,1,1
4,4,I would recommend this product because it is p...,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
test=pd.read_csv('test.csv')
test.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,Made of very thin cheap metal broke on very fi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"As good as the brand names, no jams or misfire...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"unit was easy to use, with understandable in s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,I am the new family plumber. Works well. No pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Seems to be holding up well.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
filter = train["Review"] != ""
train = train[filter]
train = train.dropna()

In [ ]:
train.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1
3,3,Work great easy to use . No issues at all with...,0,0,0,0,0,1,0,0,0,0,1,1
4,4,I would recommend this product because it is p...,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
train.shape

(6136, 14)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6136 entries, 0 to 6135
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Id                             6136 non-null   int64 
 1   Review                         6136 non-null   object
 2   Components                     6136 non-null   int64 
 3   Delivery and Customer Support  6136 non-null   int64 
 4   Design and Aesthetics          6136 non-null   int64 
 5   Dimensions                     6136 non-null   int64 
 6   Features                       6136 non-null   int64 
 7   Functionality                  6136 non-null   int64 
 8   Installation                   6136 non-null   int64 
 9   Material                       6136 non-null   int64 
 10  Price                          6136 non-null   int64 
 11  Quality                        6136 non-null   int64 
 12  Usability                      6136 non-null   int64 
 13  Pol

In [ ]:
train.columns

Index(['Id', 'Review', 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'],
      dtype='object')

In [ ]:
train_labels = train[['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity']]
train_labels.head()

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,1,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,0,0,1,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,1,0,1


**Text Cleaning**

In [ ]:
train_labels.shape[1]

12

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
!pip install emoji

     |████████████████████████████████| 170 kB 4.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169313 sha256=0f5167804c853c672b6e580a193afabca06ff07efdaf91bb0d77fcd43528b59f
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import emoji
import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
train['Review'] = train['Review'].apply(text_preprocessing_pipeline)
test['Review'] = test['Review'].apply(text_preprocessing_pipeline)

In [ ]:
X = train['Review']

In [ ]:
X

0       for some reason everybody complains and i am c...
1       i like everything about it great choice of spr...
2       excellent ceiling fan brace easy to install an...
3       work great easy to use no issues at all with h...
4       i would recommend this product because it is p...
                              ...                        
6131    easy to install just be careful with the razor...
6132    got a big pup and she had to learn about scree...
6133    we installed our nest cam system a little over...
6134    this is a below average camera the only thing ...
6135                       so easy to install amd operate
Name: Review, Length: 6136, dtype: object

In [ ]:
X1= []
sentences = list(train["Review"])
for sen in sentences:
    X1.append(sen)

y = train_labels.values

In [ ]:
y


array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1]])

In [ ]:
X1

In [ ]:
from sklearn.model_selection import StratifiedKFold,KFold,GridSearchCV,GroupKFold,train_test_split,StratifiedShuffleSplit

In [ ]:
tokenizer = Tokenizer(num_words = 100000, oov_token='<oov>')
tokenizer.fit_on_texts(X)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = max([len(x) for x in np.array(X)])

def preprocess_to_sequences(dataset, fitted_tokenizer, maxlen):
    dataset = tokenizer.texts_to_sequences(dataset)
    dataset = pad_sequences(dataset, padding="pre", truncating="pre", maxlen=maxlen)
    return dataset

In [ ]:
X_tokenized = preprocess_to_sequences(X, tokenizer, maxlen)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_tokenized, X_val_tokenized, y_train, y_val = train_test_split(X_tokenized, y, test_size=0.1)

In [ ]:
path_to_glove_file = "/content/glove.6B.300d.txt"


embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [ ]:
num_tokens = len(tokenizer.word_index) + 1
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 7033 words (1197 misses)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from tensorflow.keras.layers import Embedding
from keras import initializers
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
import tensorflow as tf
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D

In [ ]:
tf.random.set_seed(17)
model_GRU = Sequential([
    embedding_layer,
    Bidirectional(GRU(64, return_sequences=True)),
    GRU(128),
    Dense(train_labels.shape[1], activation="sigmoid")
])

In [ ]:
model_GRU.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         823100    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        63744     
 l)                                                              
                                                                 
 gru_1 (GRU)                 (None, 128)               99072     
                                                                 
 dense (Dense)               (None, 12)                1548      
                                                                 
Total params: 987,464
Trainable params: 164,364
Non-trainable params: 823,100
_________________________________________________________________


In [ ]:
model_GRU.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC"])
model_GRU.fit(X_train_tokenized, y_train, epochs=2, batch_size=200)

Epoch 1/2
28/28 [==============================] - 418s 15s/step - loss: 0.4234 - auc: 0.7725
Epoch 2/2
28/28 [==============================] - 410s 15s/step - loss: 0.3591 - auc: 0.8384


In [ ]:
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train)

# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

# vocab_size = len(tokenizer.word_index) + 1

# maxlen = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
!wget https://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2022-01-12 20:19:02--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2022-01-12 20:19:03--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.03MB/s    in 2m 42s  

2022-01-12 20:21:46 (5.06 MB/s) - ‘glove.6B.zip’ saved [862182753/862182753]



In [ ]:
!unzip '/content/glove.6B.zip'


Archive:  /content/glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [ ]:
# from numpy import array
# from numpy import asarray
# from numpy import zeros

# embeddings_dictionary = dict()

# glove_file = open('/content/glove.6B.300d.txt', encoding="utf8")

# for line in glove_file:
#     records = line.split()
#     word = records[0]
#     vector_dimensions = asarray(records[1:], dtype='float32')
#     embeddings_dictionary[word] = vector_dimensions
# glove_file.close()

# embedding_matrix = zeros((vocab_size, 100))
# for word, index in tokenizer.word_index.items():
#     embedding_vector = embeddings_dictionary.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector

In [ ]:
train_lab

6136

In [ ]:
# deep_inputs = Input(shape=(maxlen,))
# embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
# LSTM_Layer_1 = LSTM(128)(embedding_layer)
# dense_layer_1 = Dense(train_labels.shape[1], activation='sigmoid')(LSTM_Layer_1)
# model = Model(inputs=deep_inputs, outputs=dense_layer_1)

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 100)          734600    
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_2 (Dense)             (None, 12)                1548      
                                                                 
Total params: 853,396
Trainable params: 118,796
Non-trainable params: 734,600
_________________________________________________________________


In [ ]:
model_BiCon1d = Sequential([embedding_layer,
    Bidirectional(LSTM(128, return_sequences = True)),
   Conv1D(filters=128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dropout(0.2),
   Dense(train_labels.shape[1], activation='sigmoid')])

In [ ]:
model_BiCon1d.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC"])
model_BiCon1d.fit(X_train_tokenized, y_train, epochs=1, batch_size=200)

28/28 [==============================] - 783s 28s/step - loss: 0.1861 - auc: 0.9603


In [ ]:
# history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
31/31 [==============================] - 19s 602ms/step - loss: 0.3733 - acc: 2.5471e-04 - val_loss: 0.3673 - val_acc: 0.0020
Epoch 2/10
31/31 [==============================] - 19s 601ms/step - loss: 0.3731 - acc: 5.0942e-04 - val_loss: 0.3670 - val_acc: 0.0020
Epoch 3/10
31/31 [==============================] - 19s 603ms/step - loss: 0.3731 - acc: 0.0013 - val_loss: 0.3677 - val_acc: 0.0020
Epoch 4/10
31/31 [==============================] - 20s 663ms/step - loss: 0.3730 - acc: 0.0015 - val_loss: 0.3674 - val_acc: 0.0020
Epoch 5/10
31/31 [==============================] - 19s 603ms/step - loss: 0.3729 - acc: 0.0013 - val_loss: 0.3671 - val_acc: 0.0020
Epoch 6/10
31/31 [==============================] - 19s 602ms/step - loss: 0.3729 - acc: 0.0013 - val_loss: 0.3669 - val_acc: 0.0020
Epoch 7/10
31/31 [==============================] - 19s 604ms/step - loss: 0.3726 - acc: 0.0015 - val_loss: 0.3670 - val_acc: 0.0010
Epoch 8/10
31/31 [==============================] - 19s 604ms

In [ ]:
model1 = keras.models.Sequential([
    embedding_layer,
    keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True)),
    keras.layers.GRU(128),
    keras.layers.Dense(6, activation="sigmoid")
])

In [ ]:
X_test=test['Review']

In [ ]:
maxlen = max([len(x) for x in np.array(X_test)])

In [ ]:
print(maxlen)

3311


In [ ]:
test_tokenized = preprocess_to_sequences(X_test, tokenizer, maxlen)

In [ ]:
prediction_BiCon1d = model_BiCon1d.predict(test_tokenized)

In [ ]:
final_sub = pd.DataFrame(prediction_BiCon1d, columns = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'])

final_sub.shape

(2631, 12)

In [ ]:
final_sub

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0.203392,0.053054,0.015284,0.110952,0.006395,0.045447,0.026773,0.172035,0.063390,0.766873,0.010451,0.024567
1,0.007551,0.049745,0.012544,0.011622,0.010733,0.279526,0.000468,0.005254,0.014603,0.589833,0.001349,0.767152
2,0.007865,0.000458,0.008201,0.032796,0.011452,0.053116,0.006902,0.000250,0.005284,0.019509,0.986457,0.983926
3,0.022066,0.019332,0.015665,0.036735,0.048145,0.926936,0.005059,0.004603,0.027407,0.049435,0.008141,0.845489
4,0.008641,0.005382,0.006550,0.037313,0.004160,0.196070,0.015452,0.001533,0.002630,0.276992,0.004667,0.738317
...,...,...,...,...,...,...,...,...,...,...,...,...
2626,0.063375,0.000560,0.035637,0.004289,0.278740,0.021090,0.988888,0.007317,0.012499,0.570736,0.101111,0.980904
2627,0.007148,0.072660,0.532882,0.178457,0.037382,0.200754,0.075631,0.002520,0.008446,0.023874,0.002632,0.839441
2628,0.010311,0.009194,0.002156,0.002392,0.003230,0.512770,0.001491,0.032700,0.013219,0.984015,0.001162,0.713577
2629,0.001739,0.004711,0.660300,0.212873,0.010704,0.044520,0.000643,0.041564,0.018592,0.790276,0.057859,0.951228


In [ ]:
final_sub.to_csv('BiCon1d_Using_300d-2.csv',index=False)